### Modelling porto-seguero

performing classification is split in following steps:
- preprocessing: encoding (category -> One-hot encoding, ordinal -> Ordinal encoding) 
- preprocessing: normalizing 
- split train, val, test
- classifier: applying given classifier (SVC, DecisionTree, KNeighbours) with Holdout
- classifier: applying -||- with CV
- evaluation of results

below we present parameters comparison. For purpose of exersize the module utils was written which contains following functionalities:
- get_pipeline(feature_structure, clf): 
        builds a pipeline that preprocess columns specified in feature_structure and combines preprocessing part with classifier. Such built pipeline is returned
- evaluate_models(data, feature_structure, classifiers): 
        for given data, feature_structure and list of classifiers with set parameters each classifiere is trained with holdout and CV and return results for metrics we agreed upon 

In [1]:
# setup + import
from utils import *
import os
import sklearn
import pandas as pd
import pickle

# setup + import
data = pd.read_csv('../data_processed/porto-seguero.csv')
data.head()

pickle_file = open('../data_processed/porto-seguero_column_types.pkl', 'rb')
feature_structure = pickle.load(pickle_file)
pickle_file.close()


Imported datasets with feature structures:

In [2]:

feature_structure
feature_columns = feature_structure['bin'] + feature_structure['cat'] + feature_structure['cont'] + feature_structure['ord']

feature_structure["cat"]

X = data[feature_columns]
y = data['target']

values = [v  for _,v in feature_structure.items()]
# pd.DataFrame({"type":[feature_structure.keys()], "columns":  values} )
pd.DataFrame({"type":feature_structure.keys(), "columns":  values} )


,type,columns
0,bin,"[ps_ind_06_bin, ps_ind_07_bin, ps_ind_08_bin, ..."
1,cat,"[ps_ind_02_cat, ps_ind_04_cat, ps_ind_05_cat, ..."
2,ord,"[ps_ind_01, ps_ind_03, ps_ind_14, ps_ind_15, p..."
3,cont,"[ps_reg_01, ps_reg_02, ps_reg_03, ps_car_12, p..."
4,target,target


In [4]:
# for classifiers we use lists of classifiers
classifiers0 = [
    SVC(kernel='linear', random_state=42, decision_function_shape='ovo', probability=True),
    KNeighborsClassifier(n_neighbors=3, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
]

classifier1 = [ KNeighborsClassifier(n_jobs=-1)]

# used pipelines for classifiers
model1 = get_pipeline(feature_structure,classifier1[0])
model1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['ps_ind_02_cat',
                                                   'ps_ind_04_cat',
                                                   'ps_ind_05_cat',
                                                   'ps_car_01_cat',
                                                   'ps_car_02_cat',
                                                   'ps_car_03_cat',
                                                   'ps_car_04_cat',
                                                   'ps_car_05_cat',
                                                   'ps_car_06_cat',
                                                   'ps_car_07_cat',
                                                   'ps_car_09_cat',
                                                   'ps_car_10_cat',
                                                   'ps_car_11_cat']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ps_reg_01', 'ps_reg_02',
                                                   'ps_reg_03', 'ps_car_12',
                                                   'ps_car_13', 'ps_car_14',
                                                   'ps_car_15'])])),
                ('kneighborsclassifier', KNeighborsClassifier(n_jobs=-1))])

In [7]:
from sklearn.model_selection import cross_validate

# testing seprarately single classifier with holdout and cv 
result =  cross_validate(model1, X, y, cv=5, scoring=["accuracy","precision_weighted"], return_train_score=True, n_jobs=-1,  verbose=2)
result  

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 